## 선형 회귀분석

### 관련 라이브러리 호출

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import joblib
import numpy as np
import pandas as pd

In [ ]:
# 시각화 설정 모듈을 호출합니다.
from GraphicSetting import *

In [ ]:
# 통계 관련 라이브러리를 호출합니다.
from scipy import stats
import pingouin as pg

In [ ]:
# EDA 시각화 및 통계 분석 관련 모듈을 호출합니다.
import hds_stats as hds

In [ ]:
# 실수를 출력할 소수점 자릿수를 설정합니다.
%precision 3
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

### 작업 경로 확인 및 변경

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir('../data')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 출력합니다.
sorted(os.listdir())

### 실습 데이터셋 준비

In [ ]:
# z 파일을 읽고 데이터프레임 df를 생성합니다.
df = joblib.load(filename = 'Used_Cars_Price_Prep.z')

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# df의 처음 5행을 출력합니다.
df.head()

In [ ]:
# y절편 역할을 수행할 상수 1을 df의 두 번째 열로 삽입합니다.
df.insert(loc = 1, column = 'const', value = 1)

### 더미변수 생성

In [ ]:
# 범주형 입력변수로 더미변수를 생성합니다.
df = pd.get_dummies(data = df, prefix = [None], 
                    columns = ['FuelType'], 
                    drop_first = True)

In [ ]:
# df의 처음 10행을 출력합니다.
df.head(n = 10)

In [ ]:
# df의 열별 자료형을 확인합니다.
df.dtypes

In [ ]:
# 더미변수로 변환할 열이름으로 리스트를 생성합니다.
c = ['MetColor']

In [ ]:
# 지정한 변수를 정수형으로 일괄 변환합니다.
df[c] = df[c].astype(np.uint8)

In [ ]:
# df의 열별 자료형을 확인합니다.
df.dtypes

### 실습 데이터셋 분할

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn.model_selection import train_test_split

In [ ]:
# 전체 데이터의 70%를 훈련셋, 30%를 시험셋으로 분할합니다.
trSet, teSet = train_test_split(df, test_size = 0.3, random_state = 0)

In [ ]:
# 훈련셋의 목표변수 평균을 확인합니다.
trSet['Price'].mean()

In [ ]:
# 시험셋의 목표변수 평균을 확인합니다.
teSet['Price'].mean()

### 입력변수와 목표변수 분리

In [ ]:
# 목표변수명을 변수에 할당합니다.
yvar = 'Price'

In [ ]:
# 훈련셋을 목표변수 벡터와 입력변수 행렬로 분리합니다.
trReal = trSet[yvar].copy()
trSetX = trSet.drop(columns = [yvar])

In [ ]:
# 시험셋을 목표변수 벡터와 입력변수 행렬로 분리합니다.
teReal = teSet[yvar].copy()
teSetX = teSet.drop(columns = [yvar])

### 선형 회귀모형 적합 및 결과 확인

In [ ]:
# 훈련셋으로 선형 회귀모형을 적합합니다.
fit1 = hds.stat.ols(y = trReal, X = trSetX)

In [ ]:
# fit1 모형의 적합 결과를 확인합니다.
fit1.summary()

### 회귀진단: 잔차 가정 확인

In [ ]:
# fit1 모형 잔차의 등분산성 검정을 실행합니다.
hds.stat.breushpagan(model = fit1)

### 회귀진단: 잔차 그래프

In [ ]:
# fit1 모형의 잔차 그래프를 그립니다.
hds.stat.regressionDiagnosis(model = fit1)

### 회귀진단: 잔차의 정규성 검정

In [ ]:
# 잔차의 히스토그램을 그려서 분포를 확인합니다.
sns.histplot(x = fit1.resid, bins = 50, stat = 'density');

In [ ]:
# 잔차의 정규성 검정을 실행합니다.
pg.normality(data = fit1.resid)

### 영향점 확인

In [ ]:
# 훈련셋의 관측값마다 영향점 정보를 갖는 데이터프레임을 생성합니다.
aug = hds.stat.augment(model = fit1)

In [ ]:
# aug의 처음 5행을 출력합니다.
aug.head()

In [ ]:
# 훈련셋의 행 개수를 n에 할당합니다.
n = trSetX.shape[0]

In [ ]:
# 쿡의 거리가 4/n(행 개수) 초과 여부를 원소로 갖는 Out을 훈련셋에 추가합니다.
trSetX['Out'] = np.where(aug['cooksd'].gt(4/n), '1', '0')

### [참고] 영향점 시각화

In [ ]:
# Age와 Price의 산점도에 회귀직선을 추가하고 영향점의 색을 다르게 설정합니다.
sns.scatterplot(x = trSetX['Age'], y = trReal, hue = trSetX['Out'], 
                hue_order = ['0', '1'], palette = ['silver', 'red'],
                s = 15, alpha = 0.5)

sns.regplot(x = trSetX['Age'], y = trReal, 
            scatter_kws = {'alpha': 0}, 
            line_kws = {'color': 'red', 'lw': 1},
            ci = None);

### 훈련셋에서 이상치 제거

In [ ]:
# 훈련셋에서 Out의 범주별 빈도수를 확인합니다.
trSetX['Out'].value_counts()

In [ ]:
# 훈련셋에서 이상치인 행이름을 outIndex에 할당합니다.
outIndex = trSetX[trSetX['Out'].eq('1')].index
outIndex

In [ ]:
# 훈련셋에서 이상치인 행과 불필요한 열을 제거합니다.
trSetX = trSetX.drop(index = outIndex, columns = ['Out'])
trReal = trReal.drop(index = outIndex)

### 선형 회귀모형 재적합 및 결과 확인

In [ ]:
# 이상치를 제거한 훈련셋으로 선형 회귀모형을 적합합니다.
fit2 = hds.stat.ols(y = trReal, X = trSetX)

In [ ]:
# fit2 모형의 적합 결과를 확인합니다.
fit2.summary()

In [ ]:
# fit2 모형 잔차의 등분산성 검정을 실행합니다.
hds.stat.breushpagan(model = fit2)

In [ ]:
# fit2 모형의 잔차 그래프를 그립니다.
hds.stat.regressionDiagnosis(model = fit2)

### [참고] 더미변수의 시각적 이해

In [ ]:
# uelType에 따라 회귀직선을 다르게 그립니다.
labels = ['Diesel', 'Petrol']
for i, v in enumerate(labels):
    sns.regplot(data = df[df['Petrol'].eq(i)], 
                x = 'Age', y = 'Price', 
                ci = None, label = v, 
                scatter_kws = {'s': 10, 'alpha': 0.2})
plt.legend(loc = 'best', title = 'FuelType');

### 목표변수의 추정값 생성

In [ ]:
# 훈련셋으로 fit1과 fit2 모형의 추정값을 생성하고 실제값과 비교합니다.
trPred1 = fit1.predict(exog = trSetX)
trPred2 = fit2.predict(exog = trSetX)
pd.DataFrame(data = {'Real': trReal, 'Pred1': trPred1, 'Pred2': trPred2})

In [ ]:
# 시험셋으로 fit1과 fit2 모형의 추정값을 생성하고 실제값과 비교합니다.
tePred1 = fit1.predict(exog = teSetX)
tePred2 = fit2.predict(exog = teSetX)
pd.DataFrame(data = {'Real': teReal, 'Pred1': tePred1, 'Pred2': tePred2})

### 회귀모형 성능 평가

In [ ]:
# 훈련셋으로 fit1 모형의 성능지표를 출력합니다.
hds.stat.regmetrics(y_true = trReal, y_pred = trPred1)

In [ ]:
# 훈련셋으로 fit2 모형의 성능지표를 출력합니다.
hds.stat.regmetrics(y_true = trReal, y_pred = trPred2)

In [ ]:
# 시험셋으로 fit1 모형의 성능지표를 출력합니다.
hds.stat.regmetrics(y_true = teReal, y_pred = tePred1)

In [ ]:
# 시험셋으로 fit2 모형의 성능지표를 출력합니다.
hds.stat.regmetrics(y_true = teReal, y_pred = tePred2)

### [참고] MSE, MSLE, MAE, MAPE 관련 함수

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn import metrics

In [ ]:
# 시험셋 추정값으로 MSE를 출력합니다.
metrics.mean_squared_error(y_true = teReal, y_pred = tePred2)

In [ ]:
# 시험셋 추정값으로 MSLE를 출력합니다.
metrics.mean_squared_log_error(y_true = teReal, y_pred = tePred2)

In [ ]:
# 시험셋 추정값으로 MAE를 출력합니다.
metrics.mean_absolute_error(y_true = teReal, y_pred = tePred2)

In [ ]:
# 시험셋 추정값으로 MAPE를 출력합니다.
metrics.mean_absolute_percentage_error(y_true = teReal, y_pred = tePred2)

### 다중공선성 확인

In [ ]:
# 분산팽창지수를 출력하고 다중공선성 입력변수를 확인합니다.
hds.stat.vif(X = trSetX)

In [ ]:
# 다중공선성 입력변수가 있다고 가정하고 훈련셋에서 삭제합니다.
trSetX1 = trSetX.drop(columns = ['Petrol'])

In [ ]:
# 입력변수별 분산팽창지수를 다시 출력합니다.
hds.stat.vif(X = trSetX1)

### 단계적방법으로 선형 회귀모형 적합

In [ ]:
# 단계적방법으로 선형 회귀모형을 적합합니다.
fit3 = hds.stat.stepwise(y = trReal, X = trSetX, direction = 'both')

In [ ]:
# fit3 모형의 적합 결과를 확인합니다.
fit3.summary()

In [ ]:
# fit3 모형 잔차의 등분산성 검정을 실행합니다.
hds.stat.breushpagan(model = fit3)

### 표준화 회귀계수 확인

In [ ]:
# fit3 모형의 회귀계수를 출력합니다.
fit3.params

In [ ]:
# 표준화 회귀계수를 생성합니다.
beta_z = hds.stat.std_coefs(model = fit3)
beta_z

In [ ]:
# 표준화 회귀계수의 절대값을 오름차순 정렬한 결과를 출력합니다.
beta_z.abs().sort_values()

### 회귀모형 성능 평가

In [ ]:
# 시험셋으로 fit3 모형의 추정값을 생성합니다.
tePred3 = fit3.predict(exog = teSetX)

In [ ]:
# 시험셋으로 fit3 모형의 성능지표를 출력합니다.
hds.stat.regmetrics(y_true = teReal, y_pred = tePred3)

In [ ]:
# 시험셋으로 fit2 모형의 성능지표와 비교합니다.
hds.stat.regmetrics(y_true = teReal, y_pred = tePred2)

## End of Document